# Lab20: Association Analysis

In this lab we will be working on identifying *association rules*.  Association a way of identifying a combination of items that are associated with other items in the form of an IF-THEN type of rule:

    X -> Y

where X and Y are sets of items (goods, words, etc.).  The rule above asks if we purchase X then how often do we also purchase Y?

## Install mlxtend

We will be using the package `mlxtend` for our analysis.  This is a machine learning package (hence the ml) and has some features that overlap with SciKit Learn and other analysis pacakages.  

Using `Anaconda Navigator` install the `mlxtend` package.  It is on the `conda-forge` channel, so you will have to make sure that you have that set up as well.  (We did this back in Lab01.)

In [32]:
import pandas as pd

# TransactionEncoder will convert our data set to something
# the association-rule algorithm can use
from mlxtend.preprocessing import TransactionEncoder

# We will use the apriori module to generate a dataframe that
# we can use for association rule finding
from mlxtend.frequent_patterns import apriori

# We will use the association_rules module to generate
# our association rules from the apriori output data frame
from mlxtend.frequent_patterns import association_rules

## Load our groceries data set

This is from [Kaggle](https://www.kaggle.com/datasets/heeraldedhia/groceries-dataset?resource=download).  It is a CSV file containing around 38000 purchases of people from grocery stores.

Load and examine the file as a Pandas DataFrame.

Use the `sort_values` method to sort first by CustomerID and then by date, and examine the resulting sorted DataFrame. You will note that the file appears to contain daily transactions for each customer.  We will assume that a customer's daily purchases constitute a single transaction.

In [33]:
fn_grocery = 'Groceries_dataset.csv'
df_raw = pd.read_csv(fn_grocery)

In [34]:
df_raw.info()
df_raw.head()
df_raw.sort_values(['Member_number' ,'Date'],inplace=True)
df_raw

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


,Member_number,Date,itemDescription
4843,1000,15-03-2015,sausage
8395,1000,15-03-2015,whole milk
20992,1000,15-03-2015,semi-finished bread
24544,1000,15-03-2015,yogurt
13331,1000,24-06-2014,whole milk
...,...,...,...
3578,5000,10-02-2015,soda
19727,5000,10-02-2015,root vegetables
34885,5000,10-02-2015,semi-finished bread
9340,5000,16-11-2014,bottled beer


## Group the transactions

Using `groupby` group each transaction by `Member_number` and `Date`.  This will give us one "row" with each purchase per customer per day.  We will use this in a moment to encode a new DataFrame for our `mlxtend`'s use.

In [35]:
df_grouped = df_raw.groupby(['Member_number','Date'])
df_grouped

## Encode the grouped data frame

Here we need to transform our dataset with the `Member_number` and `Date` index into a DataFrame that `mlxtend` can use to calculate the support of each transaction.

The `agg` method will return a DataFrame.  We need to convert its output into a list of lists.

In [36]:
def make_transaction_list(x):
    '''
    Will return a list of the unique items
    in a particular grouping when used with
    the agg method as its function
    '''
    return x.unique()


In [37]:
# Use the agg method and make_transaction_list
# to return a list of unique items for each
# transaction
df_grouped['itemDescription'].agg(make_transaction_list) # gets data series back
df_transactions = df_grouped.agg(make_transaction_list) # gets data frame back


## Convert to list of lists

From the DataFrame you created using the `agg` function, turn the output Series into a list of lists.

In [38]:
list_trans = list(df_transactions['itemDescription'])
len(list_trans)
list_trans

[array(['sausage', 'whole milk', 'semi-finished bread', 'yogurt'],
       dtype=object),
 array(['whole milk', 'pastry', 'salty snack'], dtype=object),
 array(['canned beer', 'misc. beverages'], dtype=object),
 array(['sausage', 'hygiene articles'], dtype=object),
 array(['soda', 'pickled vegetables'], dtype=object),
 array(['frankfurter', 'curd'], dtype=object),
 array(['sausage', 'whole milk', 'rolls/buns'], dtype=object),
 array(['whole milk', 'soda'], dtype=object),
 array(['beef', 'white bread'], dtype=object),
 array(['frankfurter', 'soda', 'whipped/sour cream'], dtype=object),
 array(['frozen vegetables', 'other vegetables'], dtype=object),
 array(['butter', 'whole milk'], dtype=object),
 array(['tropical fruit', 'sugar'], dtype=object),
 array(['butter milk', 'specialty chocolate'], dtype=object),
 array(['sausage', 'rolls/buns'], dtype=object),
 array(['root vegetables', 'detergent'], dtype=object),
 array(['frozen meals', 'dental care'], dtype=object),
 array(['rolls/buns'], 

## Encode the lists

Use the [`TransactionEncoder` method](http://rasbt.github.io/mlxtend/user_guide/preprocessing/TransactionEncoder/) from `mlxtend` to encode our list of lists into a matrix of presence/absence Boolean values.  See the example at the link above.

Once you do that, we can re-convert the output into a DataFrame by creating a dataframe based on the output and the value in the TransactionEncoder's `column_` attribute.  Again, see the link above.  It shows how to do this.

In [45]:
te = TransactionEncoder()
encoded_itemset = te.fit(list_trans).transform(list_trans)
encoded_itemset
encoded_itemset.shape
te.columns_

df_encoded = pd.DataFrame(encoded_itemset,columns=te.columns_)
df_encoded.head()


milk_items = list(filter(lambda x: "MILK" in x.upper(),te.columns_))
milk_items

small_items = list(filter(lambda x: len(x) < 3, te.columns_))
small_items

df_encoded = df_encoded.drop(columns=small_items)
df_encoded
len(df_encoded.columns)

print('How many unique itmes are left?',len(df_encoded.columns))

How many unique itmes are left? 167


## Use `apriori` to get support

The input to the association rule generator is a data frame that contains support for each transaction.  The [`aprori`](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) module will do this for us.  See the link for documentation.

Let's use

    min_support=0.001  # All transactions that are > 1:1000

In [46]:
# Before we begin, let's do a small
# amount of cleanup.  Let's remove all
# columns (items) that have a name less
# than three letters.  Use the Pandas
# drop method to perform this action.
small_items = list(filter(lambda x: len(x) < 3, te.columns_))

df_encoded = df_encoded.drop(columns=small_items)

print('How many unique itmes are left?',len(df_encoded.columns))

How many unique itmes are left? 167


In [49]:
# Use apriori to create a dataframe with columns
# of support and itemset lists
df_support = apriori(df_encoded, min_support=0.001,use_colnames=True)
df_support.sort_values('support', inplace=True,ascending=False)
df_support

,support,itemsets
146,0.157923,(whole milk)
90,0.122101,(other vegetables)
109,0.110005,(rolls/buns)
123,0.097106,(soda)
147,0.085879,(yogurt)
...,...,...
344,0.001002,"(margarine, chicken)"
201,0.001002,"(chicken, bottled beer)"
202,0.001002,"(chocolate, bottled beer)"
516,0.001002,"(pastry, hamburger meat)"


## Use association_rules to find the rules

Using the dataframe generated by `apriori`, find the association rules with the greatest lift.  See the [association_rules documentation](https://rasbt.github.io/mlxtend/api_modules/mlxtend.frequent_patterns/association_rules/) for how to do this.

Sort the resulting DataFrame by lift in descending order.  A lift > 1 indicates that the items are often purchased together and that buying X will increase the purchase of Y.  A lift of < 1 indicates the items are often substituted.  That is X is substituted for Y so X and Y don't appear together often.

Examine the resulting DataFrame.  For the association rule X -> Y, X is the column `antecedents` and Y is the column `consequents`.  If sorted you can see the metrics for each rule based upon the lift.

In [52]:
# Find the association rules
rules = association_rules(df_support, metric='lift',min_threshold=1.0)

rules.sort_values('lift', ascending=False, inplace=True)

len(rules)

240

In [ ]:
# Sort the rules by lift
# and examine the output
# to find what rules were
# discovered
